# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

## Text Preprocessing
### reference
> * [Google guide](https://developers.google.com/machine-learning/guides/text-classification/step-3)
> * N-grams
>> * [scikit-learn working with text data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#)
>> * [scikit-learn text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
>> * [한글 자료](https://datascienceschool.net/03%20machine%20learning/03.01.03%20Scikit-Learn%EC%9D%98%20%EB%AC%B8%EC%84%9C%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EB%8A%A5.html)
> * Sequence
>> * [keras text classification](https://keras.io/examples/nlp/text_classification_from_scratch/)
>> * [tensorflow text classification](https://www.tensorflow.org/tutorials/keras/text_classification)

---
#### 

### 0. 라이브러리 설치 및 불러오기

In [1]:
## import sklearn
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
mecab = Mecab()
#fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


### 1.  데이터 가져오기

In [5]:
# 데이터를 가져옵니다.
import pandas as pd

df = pd.read_csv("./spam.csv")
df.head(10)

#### 1-1. processing label

In [1]:
# label 데이터를 수치형으로 변환합니다.
df.label = df.label.map({"spam" : 1, "ham" : 0})

#### 1-2. 결측치 제거

In [ ]:
df = df.dropna(axis = 0)

### 2. Train Validation(Test) Split

In [4]:
# train validation set으로 분리합니다.
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(df, test_size = 0.2, random_state=2022)

In [ ]:
print('-------------- 학습데이터의 비율 --------------')
print(f'정상의 비율 = {round(train_data.label.value_counts()[1]/len(train_data.label) * 100,3)}%')
print(f'스팸의 비율 = {round(train_data.label.value_counts()[0]/len(train_data.label) * 100,3)}%')
print('-------------- 테스트데이터의 비율 --------------')
print(f'정상의 비율 = {round(val_data.label.value_counts()[1]/len(val_data.label) * 100,3)}%')
print(f'스팸의 비율 = {round(val_data.label.value_counts()[0]/len(val_data.label) * 100,3)}%')

### 3. Vectorize texts

#### 3-1. N-grams Vectorize [참고](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#extracting-features-from-text-files)

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_train_counts = cv.fit_transform(train_data.tokenized)
x_train_counts.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(x_train_counts)
x_train_tfidf = tf_transformer.transform(x_train_counts)
x_train_tfidf.shape

#### 3-2. Sequence Vectorize [참고](https://developers.google.com/machine-learning/guides/text-classification/step-3)

In [12]:
# 불용어 지정
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

# 학습데이터 토큰화, 불용어 제거
train_data['tokenized'] = train_data.text.apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x : [item for item in x if item not in stopwords])

# 테스트데이터 토큰화, 불용어 제거
val_data['tokenized'] = val_data.text.apply(mecab.morphs)
val_data['tokenized'] = val_data['tokenized'].apply(lambda x : [item for item in x if item not in stopwords])

23


In [ ]:
import numpy as np

spam_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)
ham_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)

In [ ]:
from collections import Counter

spam_word_count = Counter(spam_words)
print(spam_word_count.most_common(20))

In [ ]:
ham_word_count = Counter(ham_words)
print(ham_word_count.most_common(20))

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
text_len = train_data[train_data['label']==0]['tokenized'].map(lambda x: len(x))
ax1.hist(text_len, color='red')
ax1.set_title('Positive Reviews')
ax1.set_xlabel('length of samples')
ax1.set_ylabel('number of samples')
print('정상 문자의 평균 길이 :', np.mean(text_len))

text_len = train_data[train_data['label']==1]['tokenized'].map(lambda x: len(x))
ax2.hist(text_len, color='blue')
ax2.set_title('Negative Reviews')
fig.suptitle('Words in texts')
ax2.set_xlabel('length of samples')
ax2.set_ylabel('number of samples')
print('스팸 문자의 평균 길이 :', np.mean(text_len))
plt.show()

In [ ]:
x_train = train_data['tokenized'].values
y_train = train_data['label'].values
x_val= val_data['tokenized'].values
y_val = val_data['label'].values

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [ ]:
# 단어 벡터 확인
tokenizer.word_index

In [ ]:
# 등장 횟수가 1회인 단어는 제거
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0                          # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0                        # 학습데이터의 전체 단어 빈도수 총합
rare_freq = 0                         # 등장 빈도수가 threshold보다 작은 단어의 빈도수의 총합

# 단어와 빈도수의 쌍을 key, value로 받음
for k, v in tokenizer.word_counts.items():
    total_freq = total_freq + v
    
    # 단어의 등장 빈도수가 threshold보다 작으면
    if(v < threshold):
        rare_cnt += 1
        rare_freq += v
        
print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)

In [ ]:
print('문자의 최대 길이 :',max(len(text) for text in x_train))
print('문자의 평균 길이 :',sum(map(len, x_train))/len(x_train))
plt.hist([len(text) for text in x_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 70
below_threshold_len(max_len, x_train)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(x_train, maxlen=max_len)
x_val = pad_sequences(x_val, maxlen=max_len)

### 4. Save data

#### 4-1. Save N-grams

In [3]:
# N-grams 방식으로 vectorize한 데이터의 shape을 확인해봅니다.
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
train_data.to_csv('./train_data.csv', encoding='utf-8', index=False)
val_data.to_csv('./val_data.csv', encoding='utf-8', index=False)

#### 4-2. Save sequence

In [15]:
# Sequence 형태로 vectorize한 데이터의 shape을 확인해봅니다.
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
pd.DataFrame(x_train.tolist()).to_csv('./x_train.csv', index=False)
pd.DataFrame(x_val.tolist()).to_csv('./x_val.csv', index=False)

X_train seq shape: (15066, 23)
X_val seq shape: (5023, 23)


#### 4-3. Save label

In [4]:
# label 데이터의 shape을 확인하고 저장합니다.
pd.DataFrame(y_train).to_csv('./y_train.csv', index=False)
pd.DataFrame(y_val).to_csv('./y_val.csv', index=False)